In [60]:
import numpy as np
import pandas as pd
import re

from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

# Load data

In [61]:
data = pd.read_csv('temp_final_data.csv')
data.head()

,name,Brand,Best Sellers Rank,weight,Laptop type,Laptop purpose,Screen Size,Hard Drive Size,Hard Drive Type,Memory Speed,...,Graphics Coprocessor,Graphics Coprocessor perf,Optical Drive Type,Operating System,Number of USB 3.0 Ports,Number of USB 2.0 Ports,Date First Available,Customer Reviews,old_price,price
0,"samsung chromebook 4 (2021 model) 11.6"" intel ...",samsung,293.0,1.179340,traditional,general,11.6,32.0,other,2.133,...,intel uhd 600,1.8,NaN,chrome os,NaN,NaN,2021-06-26,4.2,139.99,100.0
1,"lenovo - chromebook 3 11"" chromebook - amd a6 ...",lenovo,256.0,1.587573,2 in 1,general,11.6,32.0,other,NaN,...,amd radeon r2,1.0,yes,chrome os,NaN,NaN,2020-10-20,4.5,NaN,103.0
2,"newest flagship hp chromebook, 11.6"" hd (1366 ...",hp,86.0,1.356241,traditional,general,11.6,32.0,hybrid,NaN,...,intel hd 500,1.7,no,chrome os,NaN,NaN,2020-09-23,4.2,118.00,106.0
3,"samsung 11.6"" hd chromebook, intel celeron pro...",samsung,328.0,1.814369,traditional,general,11.6,32.0,ssd,2.500,...,intel hd,0.9,no,chrome os,NaN,1.0,2021-08-30,4.4,NaN,109.0
4,acer chromebook 512 laptop | intel celeron n40...,acer,15.0,1.401600,traditional,general,12.0,NaN,NaN,2.800,...,NaN,NaN,no,chrome os,4.0,NaN,2021-11-23,4.5,199.99,115.0


In [62]:
pd.Series(data.isnull().sum())[pd.Series(data.isnull().sum()) > 0].sort_values(ascending=False)

Number of USB 2.0 Ports      1136
old_price                    1109
Memory Speed                  715
Number of USB 3.0 Ports       676
Optical Drive Type            633
Customer Reviews              559
Hard Drive Type               470
Hard Drive Size               430
Graphics Coprocessor          381
Graphics Coprocessor perf     381
Memory Type                   341
Processor rank                302
Processor                     302
Memory Size                   300
Laptop type                   218
Best Sellers Rank             217
price                         149
Operating System               54
Screen Size                    45
Brand                          17
Date First Available            1
dtype: int64

# Number of USB 2.0 Ports

In [63]:
data['Number of USB 2.0 Ports'].value_counts()

1.0    241
2.0    117
3.0     17
4.0      2
Name: Number of USB 2.0 Ports, dtype: int64

Có 2 giá trị vô lý là 10 và 25 (laptop ko thể có nhiều cổng USB như v) --> loại bỏ nó (vì ko rõ thực tế như thế nào)

In [64]:
impos_ind = data[(data['Number of USB 2.0 Ports']==25) | (data['Number of USB 2.0 Ports']==10)].index
data = data.drop(impos_ind)
data['Number of USB 2.0 Ports'].value_counts()

1.0    241
2.0    117
3.0     17
4.0      2
Name: Number of USB 2.0 Ports, dtype: int64

In [65]:
fillna_arr = np.random.choice(np.arange(len(data['Number of USB 2.0 Ports'].value_counts().index)), 
                              size=len(data[data['Number of USB 2.0 Ports'].isna()]['Number of USB 2.0 Ports']), 
                              p=data['Number of USB 2.0 Ports'].value_counts().values/sum(data['Number of USB 2.0 Ports'].value_counts()))

In [66]:
def fill_specific_na(x):
    global i
    if not (pd.isnull(x)):
        return x
    i+=1
    return fillna_arr[i]

In [67]:
i=-1
data['Number of USB 2.0 Ports'] = data['Number of USB 2.0 Ports'].apply(fill_specific_na)
data['Number of USB 2.0 Ports'].value_counts()

0.0    725
1.0    584
2.0    181
3.0     21
4.0      2
Name: Number of USB 2.0 Ports, dtype: int64

Điền giá trị khiếm khuyết dựa trên tỷ lệ các giá trị.

# Number of USB 3.0 Ports

In [68]:
data['Number of USB 3.0 Ports'].value_counts().sort_index()

1.0    234
2.0    290
3.0    189
4.0    118
5.0      4
6.0      2
Name: Number of USB 3.0 Ports, dtype: int64

In [69]:
fillna_arr = np.random.choice(np.arange(len(data['Number of USB 3.0 Ports'].value_counts().sort_index())), 
                              size=len(data[data['Number of USB 3.0 Ports'].isna()]['Number of USB 3.0 Ports']), 
                              p=data['Number of USB 3.0 Ports'].value_counts().sort_index().values/sum(data['Number of USB 3.0 Ports'].value_counts().values))

In [70]:
i=-1
data['Number of USB 3.0 Ports'] = data['Number of USB 3.0 Ports'].apply(fill_specific_na)
data['Number of USB 3.0 Ports'].value_counts()

1.0    484
2.0    414
3.0    283
0.0    203
4.0    123
5.0      4
6.0      2
Name: Number of USB 3.0 Ports, dtype: int64

# old_price and price

In [71]:
data['price']

0       100.0
1       103.0
2       106.0
3       109.0
4       115.0
        ...  
1508      NaN
1509      NaN
1510      NaN
1511      NaN
1512      NaN
Name: price, Length: 1513, dtype: float64

Vì price là giá trị dự đoán --> ko thể tự ý điền giá trị thay thế --> xóa dòng.

In [72]:
nan_ind = data[data['old_price'].isna()].index
data.loc[nan_ind, 'old_price'] = data.loc[nan_ind, 'price']

In [73]:
data['old_price'].isnull().sum()

141

Vì 'old_price' là giá tiền thời điểm trước đó của laptop (giá laptop có thay đổi về sau) nên giá trị NaN tức là giá tiền laptop không thay đổi, chỉ cần gán lại đúng giá trị biến 'price' tương ứng.

# Optical Drive type

In [74]:
print(data['Optical Drive Type'].isna().sum())
data['Optical Drive Type'].value_counts()

633


no     782
yes     98
Name: Optical Drive Type, dtype: int64

In [75]:
data['Optical Drive Type'] = data['Optical Drive Type'].replace(regex=['no.*', '^(?!no).*'], value=['No', 'Yes'])
data['Optical Drive Type'].value_counts()

No     782
Yes     98
Name: Optical Drive Type, dtype: int64

In [76]:
fillna_arr = np.random.choice([0, 1], 
                              size=len(data[data['Optical Drive Type'].isna()]['Optical Drive Type']), 
                              p=data['Optical Drive Type'].value_counts().values/sum(data['Optical Drive Type'].value_counts().values))

In [77]:
i=-1
data['Optical Drive Type'] = data['Optical Drive Type'].apply(fill_specific_na).replace(to_replace=[0,1], value=['No', 'Yes'])
data['Optical Drive Type'].value_counts()

No     1358
Yes     155
Name: Optical Drive Type, dtype: int64

# Hard Drive type

In [78]:
data['Hard Drive Type'] = data['Hard Drive Type'].str.lower()
data['Hard Drive Type'].value_counts()

ssd       926
other      77
hdd        34
hybrid      6
Name: Hard Drive Type, dtype: int64

In [79]:
fillna_arr = np.random.choice(np.arange(len(data['Hard Drive Type'].value_counts().index)), 
                              size=len(data[data['Hard Drive Type'].isna()]['Hard Drive Type']), 
                              p=data['Hard Drive Type'].value_counts().values/sum(data['Hard Drive Type'].value_counts().values))

i=-1
data['Hard Drive Type'] = data['Hard Drive Type'].apply(fill_specific_na).replace(to_replace=np.arange(len(data['Hard Drive Type'].value_counts().index)), 
                                                                                        value=data['Hard Drive Type'].value_counts().index)
data['Hard Drive Type'].value_counts()

ssd       1346
other      110
hdd         49
hybrid       8
Name: Hard Drive Type, dtype: int64

Vì chênh lệch số lượng quá lớn giữa các giá trị --> điền giá trị theo tỷ lệ xuất hiện của các giá trị.

# Hard Drive size

In [80]:
data['Hard Drive Size'].isnull().sum()

430

In [81]:
data['Hard Drive Size'].value_counts()

512.0       299
256.0       251
1024.0      184
128.0       107
64.0         84
32.0         77
16.0         25
500.0        16
2048.0       12
1000.0       11
1.0           4
640.0         2
750.0         2
524288.0      2
4096.0        2
3000.0        1
320.0         1
1512.0        1
2.0           1
262144.0      1
Name: Hard Drive Size, dtype: int64

Vì dung lượng phần cứng là những con số cố định nên khi điền vào giá trị khiếm khuyết:
+ Không áp dụng phương pháp nội suy.
+ Không dùng giá trị mean.

In [82]:
data['Hard Drive Size'] = data['Hard Drive Size'].fillna(data.groupby('Hard Drive Type')['Hard Drive Size'].transform('median')).ffill()
data['Hard Drive Size'].value_counts()

512.0       685
256.0       251
1024.0      184
64.0        111
128.0       107
32.0         77
16.0         25
500.0        16
887.0        15
2048.0       12
1000.0       11
1.0           4
320.0         3
524288.0      2
4096.0        2
640.0         2
750.0         2
1512.0        1
2.0           1
3000.0        1
262144.0      1
Name: Hard Drive Size, dtype: int64

Vì dung lượng phần cứng là số cố định nên ta dùng phương thức transform của groupby pandas. Thông qua gom nhóm ta lấy giá trị trung vị điền vào giá trị khiếm khuyết.

# Memory type

In [83]:
data['Memory Type'].value_counts()

ddr4       733
lpddr4     112
lpddr4x     80
ddr3        58
ddr5        58
sdram       38
lpddr5      25
lpddr3      21
so-dimm     14
ddr3l       13
ddr         11
ddr2         3
dram         3
dimm         1
gddr6        1
a8           1
Name: Memory Type, dtype: int64

In [84]:
drop_ind = data[data['Memory Type'].isin(['A8', 'DIMM', 'GDDR6'])].index
data.drop(drop_ind, inplace=True)

In [85]:
fillna_arr = np.random.choice(np.arange(len(data['Memory Type'].value_counts().index)), 
                              size=len(data[data['Memory Type'].isna()]['Memory Type']), 
                              p=data['Memory Type'].value_counts().values/np.sum(data['Memory Type'].value_counts().values))

i=-1
data['Memory Type'] = data['Memory Type'].apply(fill_specific_na).replace(to_replace=np.arange(len(data['Memory Type'].value_counts().index)), 
                                                                            value=data['Memory Type'].value_counts().index)
data['Memory Type'].value_counts()

ddr4       958
lpddr4     145
lpddr4x     97
ddr5        76
ddr3        73
sdram       44
lpddr5      29
lpddr3      26
so-dimm     21
ddr3l       18
ddr         16
dram         4
ddr2         3
dimm         1
gddr6        1
a8           1
Name: Memory Type, dtype: int64

# Memory Size

In [86]:
data['Memory Size'].value_counts()

4.0      365
16.0     355
8.0      330
32.0      87
12.0      18
64.0      17
2.0       13
1.0        9
6.0        6
20.0       6
128.0      4
24.0       1
256.0      1
5.0        1
Name: Memory Size, dtype: int64

In [87]:
data['Memory Size'] = data['Memory Size'].fillna(data.groupby('Memory Type')['Memory Size'].transform('median'))
data['Memory Size']

0        4.0
1       32.0
2        4.0
3        4.0
4        4.0
        ... 
1508    16.0
1509     4.0
1510     8.0
1511     6.0
1512    16.0
Name: Memory Size, Length: 1513, dtype: float64

# Memory Speed

In [88]:
data['Memory Speed'] = data['Memory Speed'].fillna(data.groupby(['Memory Size', 'Memory Type'])['Memory Speed'].transform('mean')).ffill()
data['Memory Speed'].isnull().sum()

0

Dựa theo dung lượng và loại bộ nhớ mà điền giá trị khiếm khuyết bằng trị giá trung vị.

# Customer reviews

In [89]:
data['Customer Reviews'] = data['Customer Reviews'].interpolate(method='linear').round(1)
data['Customer Reviews']

0       4.2
1       4.5
2       4.2
3       4.4
4       4.5
       ... 
1508    3.2
1509    3.6
1510    4.0
1511    4.4
1512    2.5
Name: Customer Reviews, Length: 1513, dtype: float64

Sử dụng phương pháp nội suy để suy đoán các giá trị khiếm khuyết.

# Graphics Coprocessor

In [90]:
drop_ind = data[data['Graphics Coprocessor']=='1'].index
data.drop(drop_ind, inplace=True)

In [91]:
data['Graphics Coprocessor']

0                     intel uhd 600
1                     amd radeon r2
2                      intel hd 500
3                         intel hd 
4                               NaN
                   ...             
1508                            NaN
1509                  intel uhd 620
1510                            NaN
1511                            NaN
1512    nvidia geforce rtx 3070 ti 
Name: Graphics Coprocessor, Length: 1513, dtype: object

In [92]:
encode = LabelEncoder()
temp = data[['Graphics Coprocessor']].astype("str").apply(encode.fit_transform)
final = temp.where(~data.isna(), data)
final

,Graphics Coprocessor
0,42
1,5
2,24
3,22
4,NaN
...,...
1508,NaN
1509,45
1510,NaN
1511,NaN


In [93]:
imputer = KNNImputer(n_neighbors=10)
final['new'] = imputer.fit_transform(final)
final['final'] = encode.inverse_transform(final['new'].map(int))
final

,Graphics Coprocessor,new,final
0,42,42.000000,intel uhd 600
1,5,5.000000,amd radeon r2
2,24,24.000000,intel hd 500
3,22,22.000000,intel hd
4,NaN,43.010601,intel uhd 605
...,...,...,...
1508,NaN,43.010601,intel uhd 605
1509,45,45.000000,intel uhd 620
1510,NaN,43.010601,intel uhd 605
1511,NaN,43.010601,intel uhd 605


In [94]:
data['Graphics Coprocessor'] = final['final']
data['Graphics Coprocessor'].isnull().sum()

0

# Graphics Coprocessor perf

In [95]:
data['Graphics Coprocessor perf'].isna().sum()

381

In [96]:
data['Graphics Coprocessor perf'] = data.groupby('Graphics Coprocessor')['Graphics Coprocessor perf'].apply(lambda x: x.interpolate()).bfill()
data['Graphics Coprocessor perf'].isnull().sum()

0

# Processor

In [97]:
data['Processor'].value_counts()

intel core i5 1135g7    84
intel celeron n4020     66
intel core i7 1165g7    54
intel core i3 1115g4    48
intel core i7 11800h    45
                        ..
intel core i3 10100t     1
intel pentium j4205      1
intel celeron n3150      1
intel core i3 10110y     1
intel core i5 6500t      1
Name: Processor, Length: 178, dtype: int64

In [98]:
encode = LabelEncoder()
temp = data[['Processor']].astype("str").apply(encode.fit_transform)
final = temp.where(~data.isna(), data)
final

,Processor
0,63
1,NaN
2,60
3,NaN
4,63
...,...
1508,153
1509,115
1510,111
1511,NaN


In [99]:
imputer = KNNImputer(n_neighbors=10)
final['new'] = imputer.fit_transform(final)
final['final'] = encode.inverse_transform(final['new'].map(int))
final

,Processor,new,final
0,63,63.000000,intel celeron n4020
1,NaN,96.536746,intel core i5 1140g7
2,60,60.000000,intel celeron n3350
3,NaN,96.536746,intel core i5 1140g7
4,63,63.000000,intel celeron n4020
...,...,...,...
1508,153,153.000000,intel core i7 8550u
1509,115,115.000000,intel core i5 8250u
1510,111,111.000000,intel core i5 6500t
1511,NaN,96.536746,intel core i5 1140g7


In [100]:
data['Processor'] = final['final']
data['Processor'].isnull().sum()

0

# Processor rank

In [101]:
data['Processor rank'].value_counts()

748.0     84
2521.0    66
719.0     54
1131.0    48
304.0     45
          ..
970.0      1
2012.0     1
2780.0     1
1691.0     1
1354.0     1
Name: Processor rank, Length: 178, dtype: int64

In [102]:
data['Processor rank'] = data.groupby('Processor')['Processor rank'].apply(lambda x: x.interpolate()).bfill()
data['Processor rank'].isnull().sum()

0

# Best Seller rank

In [103]:
data['Best Sellers Rank'].isna().sum()

217

# Laptop type

In [104]:
data['Laptop type'].value_counts()

traditional    1160
2 in 1          135
Name: Laptop type, dtype: int64

In [105]:
fillna_arr = np.random.choice([0, 1], 
                              size=len(data[data['Laptop type'].isna()]['Laptop type']), 
                              p=data['Laptop type'].value_counts().values/sum(data['Laptop type'].value_counts().values))

In [106]:
i=-1
data['Laptop type'] = data['Laptop type'].apply(fill_specific_na).replace(to_replace=[0,1], value=['traditional', '2 in 1'])
data['Laptop type'].value_counts()

traditional    1347
2 in 1          166
Name: Laptop type, dtype: int64

# Operating System

In [107]:
data['Operating System'].value_counts()

windows      1168
chrome os     291
Name: Operating System, dtype: int64

In [108]:
data['Operating System'] = data['Operating System'].fillna('uninstalled')
data['Operating System'].value_counts()

windows        1168
chrome os       291
uninstalled      54
Name: Operating System, dtype: int64

# Screen size

In [109]:
pd.Series(data['Screen Size'].value_counts())[pd.Series(data['Screen Size'].value_counts()) > 0]

14.0    492
15.6    472
11.6    135
13.3    101
17.3     96
16.0     59
17.0     21
15.0     21
13.5     16
13.0     10
1.0       6
11.0      6
13.4      6
12.5      5
10.1      4
12.0      4
12.3      3
14.1      3
10.5      2
12.2      2
14.5      2
4.7       2
Name: Screen Size, dtype: int64

In [110]:
nan_ind = data[data['Screen Size'] < 10].index
data.drop(nan_ind, inplace=True)

In [111]:
data['Screen Size'] = data.groupby('Brand')['Screen Size'].transform('median').ffill()
data['Screen Size']

0       13.3
1       14.0
2       14.0
3       13.3
4       14.0
        ... 
1508    14.0
1509    14.0
1510    14.0
1511    14.0
1512    14.0
Name: Screen Size, Length: 1505, dtype: float64

# Brand

In [112]:
data['Brand'].isnull().sum()

16

# Delete nan rows

In [113]:
data['Date First Available'].isnull().sum()

1

In [114]:
data.dropna(subset=['price', 'Best Sellers Rank', 'Brand', 'Date First Available'], axis=0, inplace=True)

# Final check NaN

In [115]:
data.isnull().sum()

name                         0
Brand                        0
Best Sellers Rank            0
weight                       0
Laptop type                  0
Laptop purpose               0
Screen Size                  0
Hard Drive Size              0
Hard Drive Type              0
Memory Speed                 0
Memory Size                  0
Memory Type                  0
Processor                    0
Processor rank               0
Graphics Coprocessor         0
Graphics Coprocessor perf    0
Optical Drive Type           0
Operating System             0
Number of USB 3.0 Ports      0
Number of USB 2.0 Ports      0
Date First Available         0
Customer Reviews             0
old_price                    0
price                        0
dtype: int64

In [116]:
data = data.rename({'Processor': 'CPU', 'Processor rank': 'CPU rank', 
                    'Graphics Coprocessor': 'GPU', 'Graphics Coprocessor perf': 'GPU performance',
                    'Memory Type': 'RAM type', 'Memory Size': 'RAM size', 'Memory Speed': 'RAM speed'}, axis=1)

In [117]:
data['Laptop type'].value_counts()

traditional    1023
2 in 1          122
Name: Laptop type, dtype: int64

# Export tidy data

In [118]:
data.to_csv('tidy_data.csv', index=False)